In [20]:
import pandas as pd
import numpy as np
ncbirths = pd.read_csv("./Downloads/ncbirths.csv.bz2", sep="\t")

In [2]:
ncbirths.head(7)

,fage,mage,mature,weeks,premie,visits,marital,gained,weight,lowbirthweight,gender,habit,whitemom
0,NaN,13,younger mom,39.0,full term,10.0,not married,38.0,7.63,not low,male,nonsmoker,not white
1,NaN,14,younger mom,42.0,full term,15.0,not married,20.0,7.88,not low,male,nonsmoker,not white
2,19.0,15,younger mom,37.0,full term,11.0,not married,38.0,6.63,not low,female,nonsmoker,white
3,21.0,15,younger mom,41.0,full term,6.0,not married,34.0,8.00,not low,male,nonsmoker,white
4,NaN,15,younger mom,39.0,full term,9.0,not married,27.0,6.38,not low,female,nonsmoker,not white
5,NaN,15,younger mom,38.0,full term,19.0,not married,22.0,5.38,low,male,nonsmoker,not white
6,18.0,15,younger mom,37.0,full term,12.0,not married,76.0,8.44,not low,male,nonsmoker,not white


### 1: Simulations

1. Difference in average birth weight between mothers who are smokers and those who are not

In [56]:
weight_diff = ncbirths[ncbirths.habit == 'smoker'].weight.mean() - \
              ncbirths[ncbirths.habit == 'nonsmoker'].weight.mean()

weight_diff

-0.3155424644084732

In [59]:
# For the absolute value
abs(weight_diff)

0.3155424644084732

2. Mean and standard deviation of the birth weight in the data

In [22]:
mu, sigma = ncbirths.weight.agg([np.mean, np.std])

In [19]:
print('Mean = ' + str(mu) + ' and std = ' + str(sigma))

Mean = 7.101000000000028 and std = 1.5088602517328296


3. Simulation of the number of smokers and non-smokers

In [36]:
ncbirths.groupby('habit').weight.count()

habit
nonsmoker    873
smoker       126
Name: weight, dtype: int64

In [75]:
# Let's approximate the sample size for both smokers and non-smokers
smokers = np.random.normal(mu, sigma, size=130)
nonsmokers = np.random.normal(mu, sigma, size=875)
print('Sample size: smokers = ' + str(len(nonsmokers)) + ' and non-smokers = ' + str(len(smokers)))

Sample size: smokers = 875 and non-smokers = 130


4. Difference between the (simulated) mean weights for smokers and non-smokers

In [76]:
np.mean(smokers) - np.mean(nonsmokers)

-0.02216227879972532

5. Now let's repeat this process for R = 1000 times

In [77]:
R = 1000

# mean_diffs stores all the 1000 difference in means of the smokers and non-smokers samples
mean_diffs = [np.mean(np.random.normal(mu, sigma, size=130)) - 
              np.mean(np.random.normal(mu, sigma, size=875)) for i in range(R)]

# We're interested in the absolute values
mean_diffs = np.array(np.abs(mean_diffs))

6. Compute the 2.5% and 97.5% sample quantiles. Does the difference between smokers and non-smokers fall inside or outside this interval?

In [78]:
np.quantile(mean_diffs, [0.025, 0.975])

array([0.00488441, 0.32021476])

In [79]:
if (abs(weight_diff) > np.quantile(mean_diffs, [0.025, 0.975])[0] and \
    abs(weight_diff) < np.quantile(mean_diffs, [0.025, 0.975])[1]):
    print('Inside the interval!')
else:
    print('Outside the interval!')

Inside the interval!


7. The actual difference between smokers and non-smokers falls **inside** the interval. Therefore it is not statistically significant. It almost fell outside the interval though, but it was a bit below the upper bound (97.5% quantile).

### 2: t-test

1. Standard error (SE) of the mean

In [81]:
# Sample's standard deviation over the square root of the sample size
se = sigma/np.sqrt(len(ncbirths))
se

0.04771435066370764

Alternatively:

In [86]:
from scipy import stats
stats.sem(ncbirths.weight)

0.04771435066370766

2. Computing the t-value

In [109]:
# According to the equation for sigma in the 'sl05a-compare-means' pdf file on slide 33:
sigma_t_test = np.sqrt(np.var(smokers)/len(smokers) + np.var(nonsmokers)/len(nonsmokers))
sigma_t_test

0.13667860518923652

In [112]:
t_value = np.abs((np.mean(smokers) - np.mean(nonsmokers))/sigma_t_test)
t_value

0.1621488510878556

##### T Table:
![T Table!](https://lh3.googleusercontent.com/proxy/lHcbfC_mzanUuy39I_qv7cKAQodGGxif6lOknllfAL2EN_32kG57qGtQjn0kgQa6_TnHghmODej_6OL2FAtwzUhh2DVv2Zcw70LD6PXYorpfi634rrKLKrzrf7zlrUDYJOrV3BpnCA)

3. The critical t-value from the table for degrees of freedom of approximately 1000 at a two-tailed 0.05 probability (95% confidence level) is 1.962.

4. Therefore our we have a very low confidence level on our calculated t-value.